In [1]:
# import pymongo and connect Mongo client with the ipadress from the shell
import pymongo
import pandas as pd
db=pymongo.MongoClient("mongodb://127.0.0.1:27017/")

# Creating new db named 'task'
mydb=db['task']
# Creating new collection 'phonebook'
mycollection=mydb['phonebook']
#mycollection.drop()

In [2]:
class phoneDirectory:
    # __init__ method
    def __init__(self):
        print('******Phone Directory******')
     
    # method to get the user input from the Main Menu. 
    def userInput(self):
        print('----------Menu------------')
        print('Enter 1 to create new contact.')
        print('Enter 2 to update contact.')
        print('Enter 3 to search contact.')
        print('Enter 4 to delete a contact.')
        print('Enter 5 to display all contacts.')
        self.choice=input()
        
        # validating the user input
        if self.choice.isdigit() == False or int(self.choice) > 5 or int(self.choice) < 1 :
            print('\nYou have choosen Invalid menu Option. Pls retry!!\n')
            self.userInput()
        
        return int(self.choice)
    
    # method to create new contact in the phonebook directory
    def createNewContact(self):
        self.newContact={}
        
        self.newContact['_id']=input("\nEnter the new Contact's Name   :")     #Validating new UserID's Contact name.
        self.contactCheck = mycollection.find_one({'_id':self.newContact['_id']},{})
        if self.contactCheck != None:
            print(f"Contact {self.newContact['_id']} is already in Phonebook directory. Pls add new contact.")
            self.createNewContact()
            
        self.newContact['number']=input("Enter the new Contact's Number :") 
        if len(self.newContact['number']) != 10:                               #Validating new UserID's Contact number.
            print('\nInvalid mobile number. Pls re-enter with 10 digits!!')
            self.createNewContact()
        else:
            self.newContact['number'] = int(self.newContact['number'])
            self.newContact['mailid']=input("Enter the new Contact's mail id:")
            
            import re                                                          #Validating new UserID's mailid.
            self.count1=0
            self.count2=0
            self.position=None

            self.l1=re.findall('@',self.newContact['mailid'])
            self.count1 = len(self.l1)

            if self.count1==1:
                for match in re.finditer('@',self.newContact['mailid']):
                    self.position=match.start()

                self.l2=re.split('@',self.newContact['mailid'],2)

                self.count2=str(self.l2[1]).count('.')

            # inserting the new data entered from user to collection.    
            if self.count1 != 1 or self.count2 <1 or self.newContact['mailid'][self.position+1]=='.':
                print('\nInvalid Mail ID. Pls re-enter with abc@xyz.com format!!!')        
                self.createNewContact()
            else: 
                self.data1=mycollection.insert_one(self.newContact)
                print(f"\nNew Contact for {self.newContact['_id']} is added successfully")  
    
    #method to update the contact from the phonebook directory
    def updateContact(self):
        self.updateName = input("\nEnter the name of the contact to update:")
         
        self.list = mycollection.find_one({'_id':self.updateName},{})
        if self.list == None:
            print(f'{self.updateName} not in Directory. Pls try again')
            self.updateContact()
        else:
            self.newNumber  = input('Enter the new contact number to update:')
            if len(self.newNumber) != 10:
                print('\nInvalid mobile number. Pls re-enter with 10 digits!!')
                self.updateContact()
            else:
                self.newNumber = int(self.newNumber)
                self.query = {'_id':self.updateName}
                self.newValue = {'$set':{'number':self.newNumber}}
                self.data2 = mycollection.update_one(self.query,self.newValue)
                print(f"\nUpdate for Contact {self.updateName} is successfully") 
            
    # method to search the existing contact from phonebook directory
    def searchContact(self):
        self.searchData = input('Enter either name or number to search contact details:')
        # if search data is number, search the directory by number
        if self.searchData.isdigit() == True:
            self.data3 =mycollection.find_one({'number':int(self.searchData)})
            if (self.data3 != None):
                print(f'\nThe details for {self.searchData} is:\n',self.data3)
            else:
                print(f'\n{self.searchData} not in Phonebook Directory! Try Again')                  
        # if the search data is name, search the directory by name
        else:
            self.data3 = mycollection.find_one({'_id':self.searchData})
            if (self.data3 != None):
                print(f'\nThe details for {self.searchData} is:\n',self.data3)
            else:
                print(f'\n{self.searchData} not in Phonebook Directory! Try Again') 
  
    #method to delete the contact from the phonebook directory
    def deleteContact(self):
        self.deleteData = input('Enter either name or number to delete from PhoneBook Directory:')
        if self.deleteData.isdigit() == True:
            self.data1 =mycollection.delete_one({'number':int(self.deleteData)})
        else:
            self.data1 = mycollection.delete_one({'_id':self.deleteData})
        
        if self.data1.deleted_count == 1:
            print(f'\nThe details for {self.deleteData} is deleted')
        else:
            print(f'\nThe details for {self.deleteData} is not present in Directory')
            
              
    #method to display all the contact details from phonebook directory as pandas directory.           
    def displayContacts(self):
        self.data2 = mycollection.find()
       #construct the DataFrame from data2
        self.df =  pd.DataFrame(list(self.data2))
              
        print ('\nThe entire PhoneBook Directory is:\n\n',self.df)  
                      
    #Method to say Thank You
    def thankYou(self):
        self.finalChoice=input("\nEnter 'Y' to Restart again.\nEnter 'N' to Exit from Phonebook.\n")
        if self.finalChoice == 'Y' or self.finalChoice == 'N':
            return(self.finalChoice)
        else:
            print('Invalid Section. Pls enter either Y or N. Retry again!')
            self.thankYou()

In [3]:
#create an object for phoneDirectory class
p=phoneDirectory()
final='Y'

while(final=='Y'):
    #accept the user input until it is valid input
    choice=p.userInput()

    #call createNewContract method if the choice is 1
    if choice == 1:
        p.createNewContact()

    #call updateContact method if the choice is 2
    elif choice == 2:
        p.updateContact()

    #call searchContact method if the choice is 3
    elif choice == 3:
        p.searchContact()

    #call deleteContact method if the choice is 4
    elif choice == 4:
        p.deleteContact() 

    #call displayContacts method if the choice is 5
    elif choice == 5:
        p.displayContacts()

    final=p.thankYou()

******Phone Directory******
----------Menu------------
Enter 1 to create new contact.
Enter 2 to update contact.
Enter 3 to search contact.
Enter 4 to delete a contact.
Enter 5 to display all contacts.
5

The entire PhoneBook Directory is:

        _id      number            mailid
0    Name1  9980123456   Name1@gmail.com
1    Name2  1234567890   Name2@gmail.com
2    Name3  2345678901   Name3@gmail.com
3    Name4  4567890123   Name4@gmail.com
4    Name5  1234567890   Name5@gmail.com
5    Name6  1234567890   Name6@gmail.com
6    Name8  9868687978   Name8@gmail.com
7    Name9  1234567890   Name9@gmail.com
8    Name7  1234560987   Name7@gmail.com
9   Name10   987654321  Name10@gmail.com
10  Name11  9945990088  Name11@gmail.com
11  Name12  9986112233  Name12@gmail.com
12  Name13  9980776655  name13@gmail.com
13  Name20  9986123456  name20@gmail.com
14  Name21  9986009988  Name21@gmail.com

Enter 'Y' to Restart again.
Enter 'N' to Exit from Phonebook.
Y
----------Menu------------
Enter 1 to